In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd

# Read data
df = pd.read_csv('train_clean.csv', index_col='Id')

df_encoded = pd.get_dummies(df)

X = df_encoded.drop('Verkaufspreis', axis=1)
y = df_encoded['Verkaufspreis']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=42)

In [7]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import f1_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
# Create a random forest regressor
rf = RandomForestRegressor()

# Train the model
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

r2 = r2_score(y_test, y_pred)
print("R^2 Score:", r2)

# Calculate mean squared error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

R^2 Score: 0.8322866663192794
Mean Squared Error: 605269222.9944131
